# 아이펠톤에서 사용할 데이터 생성하는 노트북

### 발생한 이슈
- 필수 문장 체크하는 코드를 이곳 저곳에서 사용할 것 같음 
- class로 만들어서 재활용 가능하게 만들어봄

### 코드 흐름
1. 필수 문장 잘 포함하고 있는지 평가하는 부분의 코드 작성
    - 필수 문장 (thought)가 잘 들어 있는가? - 중복된 단어 수 + 거리
    - 생성된 여러 데이터셋에 공통된 표현이 존재하는가? - 중복 문장 비교
2. 기존에 만든 C2D2 데이터셋을 위 기준으로 변경하여 확인해보기

In [1]:
import pandas as pd
import nltk
from difflib import SequenceMatcher
import string

In [66]:
# 필수 문장이 잘 들어 있는가? 
## 필수 문장 내 단어와 story의 문장 중에 단어가 겹치는 % 비율 계산
## Longest Common Subsequence(LCS, 최장 공통 부분 문자열) 알고리즘을 사용하여 문장 유사도 측정

class instruct_check_sentence_include:
    # x = 데이터프레임
    # compare_column = 비교할 문장 컬럼명 텍스트
    # phragraph = 비교할 문단 컬럼명 텍스트
    def __init__(self, x, compare_column, phragraph):
        self.data = x
        self.data["compare_sentences"] = self.data[compare_column].str.replace(', ', '. ', case=False)
        self.data["compare_sentences"] = self.data["compare_sentences"].apply(lambda x: nltk.sent_tokenize(x))
        
        self.data["phragraph_sentences"] = self.data[phragraph].str.replace(', ', '. ', case=False)
        self.data["phragraph_sentences"] = self.data["phragraph_sentences"].apply(lambda x: nltk.sent_tokenize(x))

    ## 필수 문장 내 단어와 story의 문장 중에 단어가 겹치는 % 비율 계산
    ### 문장 부호를 제거한 후, 소문자로 변환하고 단어로 분리하는 함수
    def clean_and_split(self, sentence):
        # 문장 부호 제거 (string.punctuation을 사용하여 기본적인 문장 부호 제거)
        cleaned_sentence = sentence.translate(str.maketrans('', '', string.punctuation))
                
        # 소문자로 변환하고 단어로 분리
        words = cleaned_sentence.lower().split()
        
        return words

    ### 두 문장 간의 공통 단어 비율을 계산하는 함수
    def partial_inclusion_ratio(self, compare_sentence, phragraph) :
        # 두 문장을 단어로 분리
        compare_words = list(self.clean_and_split(compare_sentence))
        phragraph_words = list(self.clean_and_split(phragraph))

        # 공통 문자를 찾기 위해 base_sentence의 문자들이 sentence에 얼마나 포함되는지 확인
        common_words = [word for word in compare_words if word in phragraph_words]
        
        # 포함된 문자 비율을 계산
        if len(common_words) == 0:
            inclusion_ratio = 0
        else:
            inclusion_ratio = len(common_words) / len(compare_words)
        
        return round(inclusion_ratio, 2)
    
    ## Longest Common Subsequence(LCS, 최장 공통 부분 문자열) 알고리즘을 사용하여 문장 유사도 측정
    def similar(self, a, b):
        return round(SequenceMatcher(None, a, b).ratio(), 2)
    
    ### 각 row의 모든 문장 쌍에 대해 유사도를 계산하는 함수
    def calculate_all_ratios(self):
        # 새로운 리스트에 각 row에 대해 유사도를 저장
        compare_sentences_list = [] # 포함되어야 하는 문장
        phragraph_sentences_list = [] # 공통단어비율 계산할 story 문장
        row_similarity_ratios = []
        min_ratios = []
        
        phragraph_LCS_sentences_list = []
        row_LCS_similarity_ratios = []
        min_LCS_ratios = []
        
        final_p_sen_list = []
        final_max_ratio_list = []
        final_min_ratio = []
        
        for index, row in self.data.iterrows():
            compare_sentences = row['compare_sentences']
            phragraph_sentences = row['phragraph_sentences']
            
            # 각 문장 쌍에 대해 partial_inclusion_ratio 계산
            c_list = []
            p_list = []
            row_ratios = []
            LCS_p_list = []
            LCS_row_ratios = []
            final_p_list = []
            final_max_ratios = []
            
            for compare_sentence in compare_sentences:
                max_ratio = 0
                p_sen = ''
                max_LCS_ratio = 0
                LCS_p_sen = ''
                final_p_sen = ''
                final_max_ratio = 0
                
                # 문장이 너무 짧으면 비교하지 않음
                if len(compare_sentence) <= 2:
                    continue
                
                for phragraph_sentence in phragraph_sentences:
                    # 문장이 포함관계이면, 공통 단어 비율을 1로 저장
                    clean_c_sen = compare_sentence.translate(str.maketrans('', '', string.punctuation))
                    clean_p_sen = phragraph_sentence.translate(str.maketrans('', '', string.punctuation))
                    
                    if clean_c_sen in clean_p_sen or clean_p_sen in clean_c_sen:
                        max_ratio = 1.0
                        p_sen = phragraph_sentence
                        # continue
                    else:
                        # 공통 단어 비율 계산
                        ratio = self.partial_inclusion_ratio(compare_sentence, phragraph_sentence)

                        if ratio > max_ratio: 
                            max_ratio = ratio
                            p_sen = phragraph_sentence
                        
                    # LCS 관점의 유사도 계산
                    LCS_ratio = self.similar(compare_sentence, phragraph_sentence)
                    
                    # LCS 유사도가 더 높으면 업데이트
                    if LCS_ratio > max_LCS_ratio:
                        max_LCS_ratio = LCS_ratio
                        LCS_p_sen = phragraph_sentence
                        
                    # 최종 문장 저장
                    if max_ratio >= max_LCS_ratio:
                        final_p_sen = p_sen
                    else:
                        final_p_sen = LCS_p_sen
                        
                    # 최종 유사도 저장
                    if max_ratio >= max_LCS_ratio:
                        final_max_ratio = max_ratio
                    else:
                        final_max_ratio = max_LCS_ratio
                    
                c_list.append(compare_sentence) # 필수 비교 문장 저장
                p_list.append(p_sen) # 공통단어비율 문장 저장
                row_ratios.append(max_ratio) # 공통단어비율 저장
                LCS_p_list.append(LCS_p_sen) # LCS 문장 저장
                LCS_row_ratios.append(max_LCS_ratio) # LCS 저장
                final_p_list.append(final_p_sen) # 최종 문장 저장
                final_max_ratios.append(final_max_ratio) # 최종 유사도 저장
                    
            # 한 row에서 가장 높은 유사도 문장과, 가장 낮은 유사도 저장
            compare_sentences_list.append(c_list)
            phragraph_sentences_list.append(p_list)
            row_similarity_ratios.append(row_ratios)
            min_ratios.append(min(row_ratios) if row_ratios else 0)
            phragraph_LCS_sentences_list.append(LCS_p_list)
            row_LCS_similarity_ratios.append(LCS_row_ratios)
            min_LCS_ratios.append(min(LCS_row_ratios) if LCS_row_ratios else 0)
            final_p_sen_list.append(final_p_list)
            final_max_ratio_list.append(final_max_ratios)
            final_min_ratio.append(min(final_max_ratios) if final_max_ratios else 0)

        # 데이터프레임에 결과 추가
        self.data['compare_sentences'] = compare_sentences_list
        self.data['phragraph_sentences'] = phragraph_sentences_list
        self.data['공통단어비율'] = row_similarity_ratios
        self.data['min_공통단어비율'] = min_ratios
        self.data['LCS_sentences'] = phragraph_LCS_sentences_list
        self.data['LCS_유사도'] = row_LCS_similarity_ratios
        self.data['min_LCS_유사도'] = min_LCS_ratios
        self.data['final_p_sen'] = final_p_sen_list
        self.data['final_max_ratio'] = final_max_ratio_list
        self.data['final_min_ratio'] = final_min_ratio
        return self.data

In [67]:
raw = pd.read_csv("data/c2d2_0924_final.csv")
raw.head(1)

,story,Distorted part,label,Distorted_문장분리,Distorted_문장유사도,Distorted_문장유사도평균,기존데이터_scenario,기존데이터_thought
0,"I'm an introverted person, and I've just arriv...",['Are the people in this environment unfriendl...,Overgeneralization,['Are the people in this environment unfriendl...,[100.0],100.0,"I'm an introverted person, and I've just arriv...",Are the people in this environment unfriendly?


In [68]:
a = instruct_check_sentence_include(raw, '기존데이터_thought', 'story')

a.calculate_all_ratios().to_csv("data/test.csv", index=False)

### 생성된 데이터셋에 공통된 문장 표현이 존재하는가? - 중복 문장 비교

# 코드 흐름
- 완성된 데이터셋, story 컬럼 입력
- story 컬럼 내 모든 텍스트를 가져와서 문장으로 분리 (, )는 (. )으로 바꿔서 더 잘게 나눌 것
- 문장 별 count 

In [10]:
import pandas as pd
import nltk
from collections import Counter

# 데이터 프레임의 story 컬럼에서 문장을 추출하고, 빈도를 계산하는 함수
def count_common_sentences(df, story_column, clean = False):
    # 모든 story 컬럼의 텍스트를 가져옴
    all_sentences = []
    
    for story in df[story_column]:
        if clean:
            # , 를 . 로 바꿔서 문장을 더 잘게 나눔
            story = story.replace(', ', '. ')
        
        # 문장을 분리 (원문 그대로)
        sentences = nltk.sent_tokenize(story)
        
        # 모든 문장을 리스트에 추가
        all_sentences.extend(sentences)
    
    # 각 문장의 빈도 계산
    sentence_counts = Counter(all_sentences)
    
    # 빈도별로 내림차순 정렬된 결과 반환
    return sentence_counts.most_common()

# 공통 문장에서 필수 문장 제거 함수
def exclude_should_thought(common_sentences, should_sentences):
    # should_thought의 문장만 추출
    should_thought_sentences = {sentence for sentence, _ in should_sentences}
    
    # common_sentences에서 should_thought에 없는 문장만 필터링
    filtered_common_sentences = [(sentence, count) for sentence, count in common_sentences if sentence not in should_thought_sentences]
    
    return filtered_common_sentences


In [5]:
raw = pd.read_csv("data/c2d2_0924_final.csv")
raw.head(1)

,story,Distorted part,label,Distorted_문장분리,Distorted_문장유사도,Distorted_문장유사도평균,기존데이터_scenario,기존데이터_thought
0,"I'm an introverted person, and I've just arriv...",['Are the people in this environment unfriendl...,Overgeneralization,['Are the people in this environment unfriendl...,[100.0],100.0,"I'm an introverted person, and I've just arriv...",Are the people in this environment unfriendly?


In [17]:
# 필수로 포함해야 하는 문장 추출
should_thought = count_common_sentences(raw, '기존데이터_thought',False)
clean_should_thought = count_common_sentences(raw, '기존데이터_thought',True)

should_scenario = count_common_sentences(raw, '기존데이터_scenario',False)
clean_should_scenario = count_common_sentences(raw, '기존데이터_scenario',True)

# 생성한 텍스트들에서 공통 문장 추출
common_sentences = count_common_sentences(raw, 'story', False)
clean_common_sentences = count_common_sentences(raw, 'story', True)

# 생성한 텍스트 공통 문장 - 필수로 포함해야 하는 문장
filtered_common_sentences = exclude_should_thought(common_sentences, should_thought)
filtered_common_sentences = exclude_should_thought(filtered_common_sentences, should_scenario)

filtered_clean_common_sentences = exclude_should_thought(clean_common_sentences, clean_should_thought)
filtered_clean_common_sentences = exclude_should_thought(filtered_clean_common_sentences, clean_should_scenario)

In [18]:
tmp_01 = pd.DataFrame(common_sentences, columns=['gen_공통문장', 'Count'])
tmp_02 = pd.DataFrame(should_thought, columns=['thought_공통문장', 'Count'])
tmp_03 = pd.DataFrame(should_scenario, columns=['scenario_공통문장', 'Count'])
tmp_04 = pd.DataFrame(filtered_common_sentences, columns=['filtered_공통문장', 'Count'])

# 데이터프레임 위아래로 합치기
sentence_df = pd.concat([tmp_01, tmp_02, tmp_03, tmp_04], axis=1)

# 저장할 CSV 파일 이름
file_name = 'data/test2.csv'

# CSV 파일로 저장
sentence_df.to_csv(file_name, index=False)
